# Soulution

In [ ]:
import sys
from simulator import simulate_revenue, score_me
import numpy as np
from scipy.optimize import brentq, fmin_cobyla

In [ ]:
def opt_value(q_bar, A):
    if q_bar <= 0:
        return np.zeros_like(A[:, :1])

    y = 0.5 * A / q_bar
    if y.ndim < 2:
        y = y.reshape(1, -1)

    gfun = lambda C, z: 1 - np.sum(np.maximum(z - C, 0))
    C_opt = np.array([[brentq(gfun, 0, max(max(x), 0), args=(x,))
                       if gfun(0, x) < 0 else 0 for x in y]]).T

    x_opt = np.maximum(y - C_opt, 0)
    V_opt = C_opt + 0.5 * np.sum(x_opt**2, axis=-1, keepdims=True)

    return 2 * q_bar * q_bar * V_opt

In [ ]:
class BasePricePolicy(object):
    def __init__(self):
        pass

    def __call__(self, days_left, tickets_left, demand_level):
        return demand_level - 10

In [ ]:
class ApproxPricePolicy(BasePricePolicy):
    def __init__(self, demand_bins):
        super(ApproxPricePolicy, self).__init__()
        self.demand_bins = demand_bins

    def __call__(self, days_left, tickets_left, demand_level):
        tickets_left, days_left = int(tickets_left), int(days_left)
        self.compute_dp(days_left, tickets_left)

        demand_level_bin = np.digitize(demand_level, self.demand_bins) - 1

        qty = np.argmax(self.current_[demand_level_bin, :tickets_left + 1]
                        + self.value_[days_left - 1, tickets_left::-1])

        price = demand_level - qty
        return price

    def compute_dp(self, days_left, tickets_left):
        dp_computed_, tickets_left = hasattr(self, "value_"), int(tickets_left)
        if dp_computed_:
            n_days, n_tickets_p1 = self.value_.shape
            dp_computed_ = (n_days >= days_left) and (n_tickets_p1 > tickets_left)
        if dp_computed_:
            return
        
        self.value_, self.current_ = self._compute_dp(days_left, tickets_left)
    
    def _compute_dp(self, n_days, n_tickets):
        current = np.zeros((len(self.demand_bins), 1 + n_tickets), dtype=float)
        for q in range(1 + n_tickets):
            current[:, q] = (self.demand_bins - q) * q
            
        V_tilde = np.zeros((n_days, 1 + n_tickets), dtype=float)
        for t in range(1, n_days):
            for x in range(1 + n_tickets):
                V_txq = current[:, :x + 1] + V_tilde[t - 1, np.newaxis, x::-1]
                V_tilde[t, x] = np.mean(np.max(V_txq, axis=-1), axis=0)
        return V_tilde, current

In [ ]:
pricing_function = ApproxPricePolicy(np.linspace(100, 200, num=2001))

In [ ]:
simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function, verbose=True)